In [1]:
%pip install pandas scikit-learn numpy


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터셋 로드
data_a = pd.read_csv('data/imdb_movies_last_version.csv')  # 2만개 영화 정보
data_b = pd.read_csv('data/movies_metadata.csv')  # 4.5만개 영화 정보

/var/folders/t7/qdmvn4v5227c20lm4jd3098m0000gn/T/ipykernel_2509/1574716347.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_b = pd.read_csv('data/movies_metadata.csv')  # 4.5만개 영화 정보


In [3]:
data_a.head()

,Unnamed: 0,Title,Year,Genre,Rating,Description,Director,Votes,Gross,Weather
0,0,The Shawshank Redemption,(1994),Drama,9.3,"Over the course of several years, two convicts...",Frank Darabont,2741365,$28.34M,Rain
1,1,Evil Dead,(2013),Horror,6.5,"Five friends head to a remote cabin, where the...",Fede Alvarez,185688,$54.24M,Rain
2,2,Bullet Train,(2022),"Action, Comedy, Thriller",7.3,Five assassins aboard a swiftly-moving bullet ...,David Leitch,355686,$103.37M,Rain
3,3,The Suicide Squad,(2021),"Action, Adventure, Comedy",7.2,"Supervillains Harley Quinn, Bloodsport, Peacem...",James Gunn,371316,$55.82M,Rain
4,4,65,(2023),"Action, Adventure, Drama",5.4,An astronaut crash lands on a mysterious plane...,Scott Beck,42928,0,Rain


In [4]:
data_b.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
data_a.dtypes

Unnamed: 0       int64
Title           object
Year            object
Genre           object
Rating         float64
Description     object
Director        object
Votes            int64
Gross           object
Weather         object
dtype: object

In [6]:
data_b.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [7]:
# 필요한 속성 결합 (제목, 장르, 소개)
data_a['combined'] = data_a['Title'].fillna('') + ' ' + data_a['Genre'].fillna('') + ' ' + data_a['Description'].fillna('')
data_b['combined'] = data_b['title'].fillna('') + ' ' + data_b['genres'].fillna('') + ' ' + data_b['overview'].fillna('')

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_a = tfidf_vectorizer.fit_transform(data_a['combined'])
tfidf_matrix_b = tfidf_vectorizer.transform(data_b['combined'])

# 코사인 유사도 계산
cosine_similarities = cosine_similarity(tfidf_matrix_b, tfidf_matrix_a)

# 가장 유사한 영화의 인덱스 찾기
most_similar_indices = cosine_similarities.argmax(axis=1)

# 데이터셋 B에 날씨 정보 추가
data_b['weather'] = data_a['Weather'].iloc[most_similar_indices].values

# 결과 확인
print(data_b.head())

# 결과를 새로운 CSV 파일로 저장
data_b.to_csv('movies_metadata_with_weather.csv', index=False)

   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         